In [3]:
import csv
import yaml
import os
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

CSV_FILE = "Network_Data/dataset_full.csv"

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def get_numerical_columns(csv_path):
    numerical_columns = []
    with open(csv_path, mode='r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        first_row = next(reader)  # Use first row to infer types
        for col, value in first_row.items():
            if is_number(value):
                numerical_columns.append(col.upper())
    return numerical_columns

def write_yaml(columns, csv_path):
    yaml_file = os.path.splitext(csv_path)[0] + "_numerical.yaml"
    with open(yaml_file, 'w', encoding='utf-8') as f:
        yaml.dump(columns, f, sort_keys=False)
    print(f"✅ Updated YAML: {yaml_file}")

class CSVHandler(FileSystemEventHandler):
    def on_modified(self, event):
        if event.src_path.endswith(".csv"):
            print(f"Detected change in {event.src_path}")
            numerical_cols = get_numerical_columns(event.src_path)
            write_yaml(numerical_cols, event.src_path)

# Watch the directory
observer = Observer()
observer.schedule(CSVHandler(), ".", recursive=False)
observer.start()

print("🔁 Watching for CSV changes... Press Ctrl+C to stop.")
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()
observer.join()

🔁 Watching for CSV changes... Press Ctrl+C to stop.
